In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/ASR/ASR

/content/gdrive/MyDrive/ASR/ASR


In [ ]:
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 5.0 MB/s 
     |████████████████████████████████| 163 kB 72.1 MB/s 
     |████████████████████████████████| 115 kB 46.3 MB/s 
     |████████████████████████████████| 212 kB 54.5 MB/s 
     |████████████████████████████████| 127 kB 53.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 42.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simp

In [ ]:
from datasets import Audio, ClassLabel, load_dataset, load_metric
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from torch._C import device
from transformers import (
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    trainer_utils
)

In [ ]:
import json
import os
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import IPython.display as ipd
import jiwer
import librosa
import numpy as np
import pandas as pd
import random
import re
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
import torchaudio
import wandb
# from datasets import Audio, ClassLabel, load_dataset, load_metric
# from IPython.display import display, HTML
# from sklearn.model_selection import train_test_split
# from torch._C import device
# from torch.utils.data import Dataset
# from transformers import (
#     EarlyStoppingCallback,
#     Trainer,
#     TrainingArguments,
#     Wav2Vec2CTCTokenizer,
#     Wav2Vec2FeatureExtractor,
#     Wav2Vec2ForCTC,
#     Wav2Vec2Processor,
#     trainer_utils
# )

In [ ]:
wandb.login(key="f8309bfa7d3b1d6d877d79040b6d30b6dc26afcb") # add wandb API key
wandb.init(project="speech")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anish_shilpakar. Use `wandb login --relogin` to force relogin


In [ ]:
import os

# os.mkdir("/kaggle/working/ne_np_female/")

In [ ]:
# colnames = ["path", "labels"] 
# df_tsv = pd.read_csv('/content/gdrive/MyDrive/ASR/ASR/ne_np_female/transcript/line_index.tsv', sep = "\t", header = None, names = colnames)
# df_tsv.to_csv('/content/gdrive/MyDrive/ASR/ASR/ne_np_female/transcript/ne_np_female.csv', index=False)

# df_openslr = pd.read_csv('/content/gdrive/MyDrive/ASR/ASR/data/transcript/dataset_v2.csv')
# df_openslr = df_openslr[["filename","text"]]
# df_openslr.head()
# df_openslr.to_csv('/content/gdrive/MyDrive/ASR/ASR/data/transcript/dataset_v3.csv', index=False)

In [ ]:
colnames = ["path", "labels"] 
transcript_path_ne_np = '/content/gdrive/MyDrive/ASR/ASR/ne_np_female/transcript/ne_np_female.csv'
audio_path_ne_np = '/content/gdrive/MyDrive/ASR/ASR/ne_np_female/audio/'
transcript_path_openslr = '/content/gdrive/MyDrive/ASR/ASR/data/transcript/dataset_v4_15k.csv'
audio_path_openslr = '/content/gdrive/MyDrive/ASR/ASR/data/audio/'
# df = pd.read_csv(transcript_path_ne_np)
df = pd.read_csv(transcript_path_openslr, usecols=["filename","text"])
df.rename(columns={"filename": "path","text":'labels'}, inplace=True)
df["path"] = audio_path_openslr + df["path"] + ".flac"
# df["path"] = audio_path_ne_np + df["path"] + ".wav"
df.head()

,path,labels
0,/content/gdrive/MyDrive/ASR/ASR/data/audio/000...,विकास क्षेत्र लुम्बिनी
1,/content/gdrive/MyDrive/ASR/ASR/data/audio/001...,मुताविक चित्रहरूको बैध
2,/content/gdrive/MyDrive/ASR/ASR/data/audio/001...,त्यसैले शाब्दिक रूपमा
3,/content/gdrive/MyDrive/ASR/ASR/data/audio/001...,दृष्टिकोणले किराँती विशेषता
4,/content/gdrive/MyDrive/ASR/ASR/data/audio/001...,नेपालको साहित्यिक कार्यक्रमबाट


In [ ]:
TEST_RATIO = 0.90 
VAL_RATIO = 0.70

In [ ]:
train_df, test_df = train_test_split(df, random_state = 0, train_size = TEST_RATIO)
train_df, val_df = train_test_split(df, random_state = 0, train_size = VAL_RATIO)

In [ ]:
%cd /content/gdrive/MyDrive/ASR/ASR

/content/gdrive/MyDrive/ASR/ASR


In [ ]:
train_df.to_csv("/content/gdrive/MyDrive/ASR/ASR/data/wav2vec_transcript/train_15k.csv", index = False)
val_df.to_csv("/content/gdrive/MyDrive/ASR/ASR/data/wav2vec_transcript/val_15k.csv", index = False)
test_df.to_csv("/content/gdrive/MyDrive/ASR/ASR/data/wav2vec_transcript/test_15k.csv", index = False)

In [ ]:
data_files = {
    "train": "train_15k.csv",
    "validation": "val_15k.csv",
    "test": "test_15k.csv"
}

In [ ]:
train_data = load_dataset("/content/gdrive/MyDrive/ASR/ASR/data/wav2vec_transcript", data_files = data_files, split = "train")
val_data = load_dataset("/content/gdrive/MyDrive/ASR/ASR/data/wav2vec_transcript", data_files = data_files, split = "validation")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/wav2vec_transcript-7828dd98864db319/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [ ]:
def show_random_elements(dataset, num_examples = 10):
    assert num_examples <=  len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(train_data.remove_columns(["path"]), num_examples = 20)

,labels
0,त्यसबेला मोहनविक्रमतिर लागे
1,काठमाडौँसम्म आउने गरेको
2,आधारमा शेर्पाको मौलिक भाषा
3,अमरसिंहको राजनीतिक पार्टी
4,ठेकुवा र चामलबाट निर्मित
5,अधिकाशं खाम्पाहरूले
6,दुर्गापुर नेपालको सगरमाथा अञ्चलको सिरहा जिल्लाको एउटा गाउँ विकास समिति हो।
7,मानिस बहुसांस्कृतिक अस्ट्रेलियाको
8,सिद्धान्तअनुसार पैठारीमा
9,उपत्यकाबाट पूर्वतिर लागे


In [ ]:
chars_to_ignore = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["labels"] = re.sub(chars_to_ignore, '', batch["labels"]).lower() + " "
    return batch

In [ ]:
train_data = train_data.map(remove_special_characters)
val_data = val_data.map(remove_special_characters)

  0%|          | 0/10655 [00:00<?, ?ex/s]

  0%|          | 0/4567 [00:00<?, ?ex/s]

In [ ]:
show_random_elements(train_data.remove_columns(["path"]))

,labels
0,भएकाले चिकित्सकहरूले तिनै
1,क्षेत्रका स्थानीयवासीको उपचारमा
2,अङ्ग्रेजहरूले दार्जीलिङलाई नेपालबाट
3,जिष्णुगुप्तको थानकोट स्थित
4,गणेशजीको प्रतिमा टाँस्नुपर्छ
5,अहिले मानिसहरूको आयु
6,सम्म कैदीहरूलाई भेट्न
7,कृष्ण धरावासीद्वारा किच्चकबधको
8,केन्द्रीय तथा उत्तरपूर्वको
9,स्नातकोत्तर यिनले इलाहाबादको


In [ ]:
vocab_path = '/content/gdrive/MyDrive/ASR/ASR/data/vocab.json'
tokenizer = Wav2Vec2CTCTokenizer(
    vocab_path, 
    unk_token = "[UNK]", 
    pad_token = "[PAD]", 
    word_delimiter_token = "|"
)

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size = 1, 
    sampling_rate = 16000, 
    padding_value = 0.0, 
    do_normalize = True, 
    return_attention_mask = True
)

In [ ]:
processor = Wav2Vec2Processor(
    feature_extractor = feature_extractor, 
    tokenizer = tokenizer
)

In [ ]:
processor.save_pretrained("/content/gdrive/MyDrive/ASR/ASR/processor_15k")

In [ ]:
train_data[0]

{'path': '/content/gdrive/MyDrive/ASR/ASR/data/audio/b0bd10f880.flac',
 'labels': 'चितामा श्रीखण्ड चन्दनजस्तो '}

In [ ]:
val_data

Dataset({
    features: ['path', 'labels'],
    num_rows: 4567
})

In [ ]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["labels"]
    
    resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    batch["sampling_rate"] = 16000
    
    return batch

In [ ]:
train_data = train_data.map(speech_file_to_array_fn, remove_columns=train_data.column_names)
val_data = val_data.map(speech_file_to_array_fn, remove_columns=val_data.column_names)

In [ ]:
# def resample(batch):
#     batch["speech"] = librosa.resample(np.asarray(batch["speech"]), batch["sampling_rate"], 16000)
#     batch["sampling_rate"] = 16000
#     return batch

In [ ]:
# train_data = train_data.map(resample, num_proc=4)
# val_data = val_data.map(resample, num_proc=4)

In [ ]:
rand_int = random.randint(0, len(train_data) - 1)
ipd.Audio(data = np.asarray(
    train_data[rand_int]["speech"]), 
    autoplay = True, 
    rate = 16000
)

In [ ]:
rand_int = random.randint(0, len(train_data) - 1)

print("Target text:", train_data[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_data[rand_int]["speech"]).shape)
print("Sampling rate:", train_data[rand_int]["sampling_rate"])

Target text: जापानी कलाकार युकियोए चित्रकार र प्रिन्टमेकर 
Input array shape: (68512,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch, processor):
        
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate = batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
train_data = train_data.map(prepare_dataset, fn_kwargs = {"processor": processor}, remove_columns = train_data.column_names, batch_size = 8, num_proc = 4, batched = True)
val_data = val_data.map(prepare_dataset, fn_kwargs = {"processor": processor}, remove_columns = val_data.column_names, batch_size = 8, num_proc = 4, batched = True)

#0:   0%|          | 0/49 [00:00<?, ?ba/s]

#1:   0%|          | 0/49 [00:00<?, ?ba/s]

#3:   0%|          | 0/49 [00:00<?, ?ba/s]

#2:   0%|          | 0/49 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which 

#0:   0%|          | 0/17 [00:00<?, ?ba/s]

#2:   0%|          | 0/17 [00:00<?, ?ba/s]

#1:   0%|          | 0/17 [00:00<?, ?ba/s]

#3:   0%|          | 0/17 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which 

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor = processor, padding = True)

In [ ]:
wer_metric = load_metric("wer")
cer_metric = load_metric("cer", revision = "master")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    
    # cer_best = 1 - cer because load best model is considering greater value of cer for better results
    cer_best = 1 - cer
    # wandb.log({'wer':wer, 'cer':cer})
    # experiment.log_metric('wer', wer)
    # experiment.log_metric('cer', cer)
    
    return {"wer": wer, "cer": cer, "cer_best": cer_best}

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "/content/gdrive/MyDrive/ASR/ASR/wav2vec_trained_models/model_ne_np_femalev1", 
    attention_dropout = 0.1,
    hidden_dropout = 0.1,
    feat_proj_dropout = 0.0,
    mask_time_prob = 0.05,
    layerdrop = 0.1,
    gradient_checkpointing = False, 
    ctc_loss_reduction = "mean", 
    pad_token_id = processor.tokenizer.pad_token_id,
    vocab_size = len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'project_hid.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1638: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [ ]:
training_args = TrainingArguments(
  output_dir = "/content/gdrive/MyDrive/ASR/ASR/wav2vec_trained_models",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16 = True,
  save_steps = 144,
  eval_steps=144,
  logging_steps=144,
  learning_rate=3e-4,
  warmup_steps=180,
  save_total_limit = 2,
  overwrite_output_dir=True,
  gradient_checkpointing = False,
  report_to = "wandb",
  metric_for_best_model = 'cer_best',
  load_best_model_at_end=True
)

ValueError: ignored

In [ ]:
import tensorflow as tf
tf.test.is_built_with_cuda()

True

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!nvidia-smi

Sat Oct 15 07:07:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
trainer = Trainer(
    model = model,
    data_collator = data_collator,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = processor.feature_extractor,
)

Using cuda_amp half precision backend


In [ ]:
train_data

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1548
})

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1548
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1440
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate c

Step,Training Loss,Validation Loss,Wer,Cer,Cer Best
144,6.911300,3.403597,1.000000,1.000000,0.000000
288,3.397300,3.130925,1.000000,1.000000,0.000000
432,1.678800,0.624511,0.694419,0.177321,0.822679
576,0.619400,0.407999,0.498425,0.111445,0.888555
720,0.422300,0.354512,0.458371,0.109131,0.890869
864,0.309800,0.340006,0.408866,0.093158,0.906842
1008,0.239200,0.333810,0.367012,0.079302,0.920698
1152,0.209000,0.326797,0.366112,0.080426,0.919574
1296,0.186600,0.328846,0.358911,0.080360,0.919640
1440,0.168300,0.330840,0.353960,0.078409,0.921591


***** Running Evaluation *****
  Num examples = 516
  Batch size = 8
Saving model checkpoint to /content/gdrive/MyDrive/ASR/ASR/wav2vec_trained_models/checkpoint-144
Configuration saved in /content/gdrive/MyDrive/ASR/ASR/wav2vec_trained_models/checkpoint-144/config.json
Model weights saved in /content/gdrive/MyDrive/ASR/ASR/wav2vec_trained_models/checkpoint-144/pytorch_model.bin
Feature extractor saved in /content/gdrive/MyDrive/ASR/ASR/wav2vec_trained_models/checkpoint-144/preprocessor_config.json
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "
***** Running Evaluation *****
  Num exampl

TrainOutput(global_step=1440, training_loss=1.4141920222176445, metrics={'train_runtime': 6045.921, 'train_samples_per_second': 7.681, 'train_steps_per_second': 0.238, 'total_flos': 7.200087718487112e+18, 'train_loss': 1.4141920222176445, 'epoch': 29.99})

In [ ]:
trainer.save_model("/content/gdrive/MyDrive/ASR/ASR/wav2vec_trained_models")

Saving model checkpoint to /content/gdrive/MyDrive/ASR/ASR/wav2vec_trained_models
Configuration saved in /content/gdrive/MyDrive/ASR/ASR/wav2vec_trained_models/config.json
Model weights saved in /content/gdrive/MyDrive/ASR/ASR/wav2vec_trained_models/pytorch_model.bin
Feature extractor saved in /content/gdrive/MyDrive/ASR/ASR/wav2vec_trained_models/preprocessor_config.json


test

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("/content/gdrive/MyDrive/ASR/ASR/wav2vec_trained_models/model_ne_np_femalev1").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("/content/gdrive/MyDrive/ASR/ASR/processor")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
input_dict = processor(val_data["input_values"][0], return_tensors = "pt", padding = True)

logits = model(input_dict.input_values.to("cuda")).logits

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
pred_ids = torch.argmax(logits, dim = -1)[0]

In [ ]:
test_transcription_data = load_dataset("/content/gdrive/MyDrive/ASR/ASR/ne_np_female/transcript", data_files = data_files, split = "test")

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(test_transcription_data["labels"][0].lower())

Prediction:
कालोटाउके गङगा चील नेपालमा पाइने एक प्रकारको चराको नाम हो

Reference:
कालोटाउके गंगाचील नेपालमा पाइने एक प्रकारको चराको नाम हो


In [ ]:
input_dict = processor(val_data["input_values"][3], return_tensors = "pt", padding = True)
print(input_dict)

logits = model(input_dict.input_values.to("cuda")).logits
print(logits)

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


{'input_values': tensor([[0.0008, 0.0008, 0.0008,  ..., 0.0008, 0.0008, 0.0008]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}
tensor([[[-0.4658, -0.3296, -2.9032,  ..., -0.7714, -6.8592, 12.2530],
         [-0.4680, -0.3152, -2.9366,  ..., -0.8317, -6.8732, 12.3143],
         [-0.4648, -0.2898, -2.9039,  ..., -0.8486, -6.8883, 12.3328],
         ...,
         [ 0.3744, -0.3150, -2.8307,  ..., -1.0509, -6.9534, 12.4945],
         [ 1.5442, -0.6042, -2.9129,  ..., -1.1653, -6.9635, 12.4679],
         [11.0722, -0.3818, -1.5613,  ..., -1.8593, -3.2529,  4.2605]]],
       device='cuda:0', grad_fn=<ViewBackward0>)


In [ ]:
pred_ids = torch.argmax(logits, dim = -1)[0]

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(test_transcription_data["labels"][3].lower())

Prediction:
फेसन पारखी उनिफेसनकोक्षेत्रमा एक आइकन थिए

Reference:
फेसन पारखी उनी फेसनको क्षेत्रमा एक आइकन थिए


Load Model for testing

In [ ]:
import librosa 

In [ ]:
def segmentLargeArray(inputTensor,chunksize=200000):
    # print(inputTensor)
    list_of_segments = []
    tensor_length = inputTensor.shape[1]
    for i in range(0,tensor_length+1,chunksize):
        list_of_segments.append(inputTensor[:,i:i+chunksize])
    return list_of_segments 

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("/content/gdrive/MyDrive/ASR/ASR/wav2vec_trained_models/model_ne_np_femalev1").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("/content/gdrive/MyDrive/ASR/ASR/processor")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
test_audio_ip1 = '/content/gdrive/MyDrive/ASR/ASR/ne_np_female/rec2.wav'
test_audio_ip2 = '/content/gdrive/MyDrive/ASR/ASR/ne_np_female/audio/nep_0258_0119737288.wav'
path = "/content/gdrive/MyDrive/ASR/ASR/ne_np_female/testy.flac"
test1 = '/content/gdrive/MyDrive/ASR/ASR/ne_np_female/test1_anjan.wav'
test2 = '/content/gdrive/MyDrive/ASR/ASR/ne_np_female/rec20sec.wav'
test3 = '/content/gdrive/MyDrive/ASR/ASR/ne_np_female/rec13sec.wav'

In [ ]:
def predict_from_speech(file):
    speech_array, sampling_rate = torchaudio.load(file)
    # print(speech_array,sampling_rate)
    resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
    resampled_array = resampler(speech_array).squeeze()
    if len(resampled_array.shape) == 1:
        resampled_array = resampled_array.reshape([1,resampled_array.shape[0]])
    # print(resampled_array.shape[1])
    if resampled_array.shape[1] >= 200000:
        print('The input file is longer than 10 seconds')
        list_of_segments = segmentLargeArray(resampled_array)
        # print(list_of_segments)
        output = ''
        for segment in list_of_segments:
            logits = model(segment.to("cuda")).logits
            pred_ids = torch.argmax(logits,dim=-1)[0]
            output += processor.decode(pred_ids)
        print(f"Prediction:\n{output}")
    else:
        print('The input file is less than 10 seconds')
        logits = model(resampled_array.to("cuda")).logits
        # print(logits)
        pred_ids = torch.argmax(logits, dim = -1)[0]
        print("Prediction:")
        print(processor.decode(pred_ids))

In [ ]:
# predict_from_speech(test_audio_ip1)
predict_from_speech(test2)
predict_from_speech(test3)

The input file is longer than 10 seconds
Prediction:
गणोदोईलोमा मठ्तेर कानेर नारचगनक्नयकुतनुजुरीको  चानआयोकभन्चक ए्तलाई उके धोेत्रीय चोत्दैनोनिर्वाचन आयोकमलेए प्रचार प्रसार का  करममाचथाववि नियाकोलार नवर्नभूुमेडवारर लाई तन सचत गराएको छ
The input file is longer than 10 seconds
Prediction:
ुघुमुघु मामग साता गयविसनबाडी पहिरो गटगारण आत सवय दीष परिवार मुनारू पविस्था पिपएगो जिल्ला पपटटटटार ट्योस्तापन समितिय जनाएको छ


In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 3            |        cudaMalloc retries: 3         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   13292 MB |   13381 MB |   22374 MB |    9082 MB |\n|       from large pool |   13286 MB |   13376 MB |   22367 MB |    9081 MB |\n|       from small pool |       5 MB |       5 MB |       6 MB |       0 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |   13292 MB |   13381 MB |   22374 MB |    9082 MB |\n|       from large pool |   13286 MB |   13376 MB |

In [ ]:
import gc
del model
gc.collect()
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 |  0% | 99% |


In [ ]:
!pip install GPUtil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=cbdf0ab63187378e9e532b1fa2501d06b40b16b8f6c2fe9ae10c0ebc4b9ec43e
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [ ]:
from GPUtil import showUtilization as gpu_usage
print("Initial GPU Usage")
gpu_usage()  
print("GPU Usage after emptying the cache")
torch.cuda.empty_cache()
gpu_usage()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  9% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% |  9% |


In [ ]:
!nvidia-smi

Mon Oct 17 08:22:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |  14964MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
path = "/content/gdrive/MyDrive/ASR/ASR/ne_np_female/testy.flac"
predict_from_speech(path)

a


RuntimeError: ignored